In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths for training and testing data
train_data_dir = '/kaggle/input/autistic-children-emotions-dr-fatma-m-talaat/Autistic Children Emotions - Dr. Fatma M. Talaat/Train'
test_data_dir = '/kaggle/input/autistic-children-emotions-dr-fatma-m-talaat/Autistic Children Emotions - Dr. Fatma M. Talaat/Test'

# Define image size and batch size
img_size = (48, 48)
batch_size = 25

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

2024-06-06 03:23:48.747450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 03:23:48.747572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 03:23:48.914208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Load test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 608 images belonging to 6 classes.
Found 150 images belonging to 6 classes.
Found 75 images belonging to 6 classes.


In [7]:
# Use pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the pre-trained model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [9]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 401ms/step - accuracy: 0.0939 - loss: 2.4380 - val_accuracy: 0.4000 - val_loss: 1.5760
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 381ms/step - accuracy: 0.3212 - loss: 1.7171 - val_accuracy: 0.4667 - val_loss: 1.4943
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 372ms/step - accuracy: 0.3977 - loss: 1.5648 - val_accuracy: 0.4333 - val_loss: 1.4640
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 365ms/step - accuracy: 0.3701 - loss: 1.6680 - val_accuracy: 0.4467 - val_loss: 1.4686
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 364ms/step - accuracy: 0.4123 - loss: 1.5607 - val_accuracy: 0.4533 - val_loss: 1.4651
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 377ms/step - accuracy: 0.4011 - loss: 1.5082 - val_accuracy: 0.4800 - val_loss: 1.4517
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 361ms/step - accuracy: 0.4398 - loss: 1.5480 - val_accuracy: 0.4600 - val_loss: 1.4598
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 361ms/step - accuracy: 0.4156 - loss: 1.5151 - val_accuracy: 0.460

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to the image you want to test
image_path = '/kaggle/input/autistic-children-emotions-dr-fatma-m-talaat/Autistic Children Emotions - Dr. Fatma M. Talaat/Test/fear/13.jpg'

# Load and preprocess the image
img = image.load_img(image_path, target_size=(48, 48))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the pixel values to be between 0 and 1

# Make predictions
predictions = model.predict(img_array)

# Get the emotion label with the highest probability
emotion_labels = ['Surprise', 'Delight', 'Sadness', 'Fear', 'Joy', 'Anger']
predicted_emotion = emotion_labels[np.argmax(predictions)]

# Print the predicted emotion
print(f'The predicted emotion is: {predicted_emotion}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
The predicted emotion is: Fear
